In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

import os
import sys
import glob
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
from IPython.display import display
from pathlib import Path
from smart_open import open
from tqdm.auto import tqdm
tqdm.pandas()

import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

import spacy
from tokenizer import MergedTokenizer

In [2]:
nlp = spacy.load('en_core_web_lg')

In [4]:
news_data = []
for cat in tqdm(os.listdir('bbc')):
    for fname in glob.glob('bbc/' + cat + '/*.txt'):
        file = Path(fname)
        iid = cat + '-' + file.name.split('.')[0]
        try:
            txt = file.read_text(encoding='utf', errors='ignore')
        except:
            print(iid)
        news_data.append([iid, txt, cat])
        
df = pd.DataFrame(news_data, columns=['id', 'text', 'category']); df

,id,text,category
0,business-001,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,business-002,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,business-003,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,business-004,High fuel prices hit BA's profits\n\nBritish A...,business
4,business-005,Pernod takeover talk lifts Domecq\n\nShares in...,business
5,business-006,Japan narrowly escapes recession\n\nJapan's ec...,business
6,business-007,Jobs growth still slow in the US\n\nThe US cre...,business
7,business-008,"India calls for fair trade rules\n\nIndia, whi...",business
8,business-009,Ethiopia's crop production up 24%\n\nEthiopia ...,business
9,business-010,Court rejects $280bn tobacco case\n\nA US gove...,business


In [70]:
merged_tokenizer = MergedTokenizer(nlp, merge_chunks=False, merge_ner=False)
tagged = []

for row in tqdm(list(df.itertuples())[:]):
    tokens = merged_tokenizer.lemmatized(row.text)
    tagged.append(TaggedDocument(words=tokens, tags=[row.Index, row.category]))

In [86]:
d2v = Doc2Vec(documents=tagged, vector_size=200, window=3, min_count=2, epochs=30, workers=4)
d2v.init_sims(replace=False)

In [87]:
def get_proba(category, topn=100):
    vec = d2v.docvecs[category]
    simdocs = d2v.docvecs.most_similar(positive=[vec], topn=topn)
    ids = list(dict(simdocs).keys())
    [ids.remove(cat) for cat in df.category.unique() if cat in ids]
    return sum(df.iloc[ids].category==category)/len(df.iloc[ids])

In [69]:
get_proba('business'), get_proba('politics'), get_proba('tech'), get_proba('entertainment'), get_proba('sport')

(0.34375,
 0.4166666666666667,
 0.336734693877551,
 0.5714285714285714,
 0.5656565656565656)

In [88]:
get_proba('business'), get_proba('politics'), get_proba('tech'), get_proba('entertainment'), get_proba('sport')

(0.34375,
 0.3684210526315789,
 0.3333333333333333,
 0.5204081632653061,
 0.45454545454545453)